<a href="https://colab.research.google.com/github/mrdyadyunov/Deep-Learning/blob/main/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить любое из двух заданий

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

Необходимо написать функции `train_one_epoch`, `train` и `predict` по шаблонам ниже (во многом повторяют примеры с семинаров).Обратите особое внимание на функцию `predict`: она должна возвращать список лоссов по всем объектам даталоадера, список предсказанных классов для каждого объекта из даталоалера и список настоящих классов для каждого объекта в даталоадере (и именно в таком порядке).

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам. Если вы выполнили оба задания, то берется максимум из двух оценок.

__Бонус__. Вы получаете 5 бонусных баллов если справляетесь с обоими заданиями на 10 баллов (итого 15 баллов). В противном случае выставляется максимальная из двух оценок и ваш бонус равен нулю.

__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [1]:
!pip install numpy torch torchvision pytorch-ignite
import pandas as pd
import numpy as np
from numpy import argmax
import torch
import torchvision
import tqdm
from tqdm.auto import tqdm
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torchvision import transforms as T
import random
import zipfile
from google.colab import drive
from ignite.metrics import Accuracy, Loss
import sys
from sklearn.metrics import accuracy_score
from torchvision.datasets import ImageFolder

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
with zipfile.ZipFile('/content/drive/MyDrive/dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

In [4]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

### Подготовка данных

In [5]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform):
        # YOUR CODE
        pass
    
    def __getitem__(self, idx):
        # YOUR CODE
        pass
    
    def __len__(self):
        # YOUR CODE
        pass

In [78]:
train_transform = T.Compose([
                T.Resize(256), 
                T.CenterCrop(224), 
                T.RandomHorizontalFlip(),
                T.ToTensor(), 
                T.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225])])
val_transform = T.Compose([
                T.Resize(256), 
                T.CenterCrop(224),
                T.RandomHorizontalFlip(),
                T.ToTensor(), 
                T.Normalize(mean=[0.485, 0.456, 0.406], 
                            std=[0.229, 0.224, 0.225])])

train_dataset = ImageFolder("/content/dataset/dataset/dataset/train", transform=train_transform)
val_dataset = ImageFolder("/content/dataset/dataset/dataset/val", transform=val_transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle = True,num_workers=1)
val_dataloader = torch.utils.data.DataLoader(val_dataset,batch_size=64,shuffle = False,num_workers=1)

In [79]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Вспомогательные функции, реализация модели

In [72]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, scheduler, device="cuda:0"):
    model = model.to(device).train()
    running_loss = np.array([])
    accs = np.array([])
    with tqdm(total=len(train_dataloader), file=sys.stdout) as prbar:
      for image, label in train_dataloader:
        image = image.to(device)
        label = label.to(device)
        label_pred = model(image)
        loss = criterion(label_pred, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()
        np.append(running_loss, loss.item())
        accuracy = (label_pred.argmax(1) == label).float().mean()
        np.append(accs, accuracy.item())
        prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
        prbar.update(1)
    metrics = {'loss': running_loss.mean(), 
               'accuracy': accs.mean()}

    print(metrics)
    return running_loss, accs


def predict(model, val_dataloder, criterion, device="cuda:0"):
    model.to(device).eval()
    losses = np.array([])
    predicted_classes = np.array([])
    true_classes = np.array([])
    with tqdm(total=len(val_dataloader), file=sys.stdout) as prbar:
        for image, label in val_dataloader:
          image=image.to(device)
          label=label.to(device)
          predicted = model(image)
          loss = criterion(predicted, label)
          np.append(losses, loss.item())
          np.appnd(predicted, predicted_classes)
          np.append(label, true_classes)
          accuracy = (predicted.argmax(1) == label).float().mean()
          prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )

    return losses, predicted_classes, true_classes


def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=10, scheduler=None):
    model.to(device)
    all_train_losses = np.array([])
    all_val_losses = np.array([])
    for epoch in range(n_epochs):
        print('Epoch', epoch+1)
        train_loss, train_accs = train_one_epoch(model, train_dataloader, 
                                                    criterion, optimizer, scheduler, device)
        np.append(all_train_losses, train_loss)
        with torch.no_grad:
          val_loss, val_preds, labels = predict(model, val_dataloader, criterion, device)
          np.append(all_val_losses, val_loss)
        pass

### Обучение модели, запуски экспериментов

In [81]:
model = nn.Sequential(    
 nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1, stride=1),
 nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 nn.ReLU(inplace=True),
 nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 nn.ReLU(inplace=True),
 nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 nn.ReLU(inplace=True),
 nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 nn.ReLU(inplace=True),
 nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 nn.ReLU(inplace=True),
 nn.MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False),
 nn.Flatten(start_dim=1, end_dim=-1),
 nn.Linear(2048, 200, bias=True)   
)

In [82]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.00002)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 0.0001, epochs=5, 
                                                steps_per_epoch=len(train_dataloader))
n_epochs = 5
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [83]:
train(model, train_dataloader, val_dataloader, criterion, optimizer, n_epochs=1, scheduler=scheduler)

RuntimeError: ignored

In [77]:
torch.cuda.empty_cache()

Простой тест на проверку правильности написанного кода

In [46]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")

RuntimeError: ignored

Запустить обучение можно в ячейке ниже.

In [ ]:
train(model, train_dataloader, val_dataloader, criterion, optimizer, device, n_epochs, scheduler)

Epoch 1


  0%|          | 0/782 [00:00<?, ?it/s]

RuntimeError: ignored

### Проверка полученной accuracy

После всех экспериментов которые вы проделали, выберите лучшую из своих моделей, реализуйте и запустите функцию `evaluate`. Эта функция должна брать на вход модель и даталоадер с валидационными данными и возврашать accuracy, посчитанную на этом датасете.

In [ ]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(true_labels, predicted_labels)
print("Оценка за это задание составит {} баллов".format(min(5, 5 * accuracy / 0.44)))

### Отчёт об экспериментах 

текст писать тут